In [1]:
import json
  
# Opening JSON file
f = open('intents.json',)
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
# Iterating through the json
# list
for i in data['intents']:
    print(i)
  
# Closing file
f.close()

{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up'], 'responses': ['Hello!'], 'context_set': ''}
{'tag': 'goodbye', 'patterns': ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day'], 'responses': ['Goodbye!'], 'context_set': ''}
{'tag': 'what', 'patterns': ['what are you', 'who are you', 'what do you do', 'what is your purpose', 'what is this'], 'responses': ['I am a USCIS chat bot for handeling questions and customer service'], 'context_set': ''}
{'tag': 'USCIS', 'patterns': ['what is USCIS', 'USCIS dose what?', 'whats USCIS do?'], 'responses': ["U.S. Citizenship and Immigration Services is an agency of the United States Department of Homeland Security that administers the country's naturalization and immigration system"], 'context_set': ''}
{'tag': 'DACA', 'patterns': ['What is purpose of DACA?', 'What is deferred action?', 'What is DACA?'], 'responses': ['Under existing regulations, an individual whose case has 

In [2]:
questions =[]
answers = []

for i in data['intents']:
    questions.append(i['patterns'])
    answers.append(i['responses'][0])
    
print(questions)
print('')
print('')
print(answers)

[['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up'], ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day'], ['what are you', 'who are you', 'what do you do', 'what is your purpose', 'what is this'], ['what is USCIS', 'USCIS dose what?', 'whats USCIS do?'], ['What is purpose of DACA?', 'What is deferred action?', 'What is DACA?'], ['Why is DACA ending?', 'Why is DACA phasing out', 'How can I get more DACA information?', 'Frequently asked DACA questions'], ['when are you open', 'what are your hours', 'hours of operation', 'what is your availability']]


['Hello!', 'Goodbye!', 'I am a USCIS chat bot for handeling questions and customer service', "U.S. Citizenship and Immigration Services is an agency of the United States Department of Homeland Security that administers the country's naturalization and immigration system", 'Under existing regulations, an individual whose case has been deferred is eligible to receive employment authorization for the 

In [3]:
# set up parrot to do parra phrases 
# Import libraries
from parrot import Parrot
import torch
import warnings
warnings.filterwarnings("ignore")
print('parrot loaded ready to work')


parrot loaded ready to work


In [4]:
##Init models (make sure you init ONLY once if you integrate this to your code)
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5", use_gpu=False)

In [12]:
#build compresed method 
def get_para(phrase, num, length):
    texts = []
    para_phrases = parrot.augment(input_phrase=phrase, use_gpu=False, diversity_ranker='levenshtein', do_diverse=False,
                                max_return_phrases=num, max_length=length, adequacy_threshold=0.9, fluency_threshold=0.9)
    for para_phrase in para_phrases:
        text = para_phrase[0]
        texts.append(text)
    
    return texts
        

text = 'Whats the most delicious papayas?'
texts = get_para(text, 10, len(text)+5)
print(texts)

['tell me the best papaya?', 'list some of the best papayas?', 'which is the best papaya?', "what's the best papaya?", 'which are the best papayas?', 'recommend some of the most delicious papayas?']


In [18]:
# build logic model to evalutae what is the question versus the answer
def text_count(s0, s1):
    s0 = s0.lower()
    s1 = s1.lower()
    s0List = s0.split(" ")
    s1List = s1.split(" ")
    return len(list(set(s0List)&set(s1List)))

#test method
#text_count('this dog is nice', 'this dog is bad')

def logic_model(text):
    index =0
    count =0
    target = -1
    for x in questions:
        for y in x:
            countz = text_count(text.lower(), y.lower())
            if(countz > count):
                count = countz
                target = index
        index +=1
    
    if(target >-1):
        return answers[target]
    else:
        return 'Sorry i do not understand please ask again'


logic_model('Hello')
        

'Sorry i do not understand please ask again'

In [13]:
# now run all intents and make 0 succsess 1 fail
results = []

index =0
for x in questions:
    answer = answers[index]
    for i in x:
        texts = get_para(i, 10, len(text)+5)
        texts.append(i)
        print(texts)
        for text in texts:
            res = logic_model(text)
            #print('AI :', res)
            #print('answer :', answer)
            if(answer == res):
                results.append(0)
            else:
                results.append(1)
    index +=1
    print(results)

print('')
print('these are the final results')
print(results)

['Hi', 'Hi']
['how are you?', 'How are you']
["whats your idea about who's there?", 'are they around yet?', 'is there anyone here?', 'is there anyone there?', 'is anyone there?', 'Is anyone there?']
['Hello', 'Hello']
['greetings', 'good day', 'Good day']
['what do you think about that', 'what happened?', "what's going on?", 'what the hell', "what's up?", "what's up", 'Whats up']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0]
['cya', 'cya']
['see you later', "we'll see you later", 'See you later']
['good bye', 'goodbye', "let's say goodbye", 'say goodbye', 'Goodbye']
["i'll leave here", "i'm leaving", 'I am Leaving']
['have a good day', 'Have a Good day']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]
["what're you doing?", 'what are you doing?', 'what do you do?', 'what are you doing', 'what is it about you?', 'what are you?', 'what are you']
['who are you?', 'who are you']
["what you're going to do?"

## Save the data to CSV

now we take the results and save them to a csv format 



In [14]:
import csv

header = ['index', 'pred', 'expected']
data = []

index = 0
for x in results:
    item = [index,x,0]
    data.append(item)
    index +=1
    
    

with open('csv data/baseline_data.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(data)

print('done all data writen to csv')

done all data writen to csv
